In [2]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# author : zlq16
# date   : 2018/4/2
import numpy as np
import pandas as pd


def main():
    S = ("S1", "S2", "S3", "S4", "S5")
    A = ("玩", "退出", "学习", "发表", "睡觉")
    strategy = pd.DataFrame(data=None, index=S, columns=A)
    reward = pd.DataFrame(data=None, index=S, columns=A)
    gama = 1
    strategy.loc["S1", :] = np.array([0.9, 0.1, 0, 0, 0])
    strategy.loc["S2", :] = np.array([0.5, 0, 0.5, 0, 0])
    strategy.loc["S3", :] = np.array([0, 0, 0.8, 0, 0.2])
    strategy.loc["S4", :] = np.array([0, 0, 0, 0.4, 0.6])
    strategy.loc["S5", :] = np.array([0, 0, 0, 0, 0])
    print("策略")
    print(strategy)
    reward.loc["S1", :] = np.array([-1, 0,   0,  0, 0])
    reward.loc["S2", :] = np.array([-1, 0,  -2,  0, 0])
    reward.loc["S3", :] = np.array([0,  0,  -2,  0, 0])
    reward.loc["S4", :] = np.array([0,  0,   0, 10, 1])
    reward.loc["S5", :] = np.array([0, 0, 0, 0, 0])
    print("回报函数")
    print(reward)


if __name__ == '__main__':
    main()

策略
      玩   退出   学习   发表   睡觉
S1  0.9  0.1  0.0  0.0  0.0
S2  0.5  0.0  0.5  0.0  0.0
S3  0.0  0.0  0.8  0.0  0.2
S4  0.0  0.0  0.0  0.4  0.6
S5    0    0    0    0    0
回报函数
     玩 退出  学习  发表 睡觉
S1  -1  0   0   0  0
S2  -1  0  -2   0  0
S3   0  0  -2   0  0
S4   0  0   0  10  1
S5   0  0   0   0  0


In [3]:
"""
A simple example for Reinforcement Learning using table lookup Q-learning method.
An agent "o" is on the left of a 1 dimensional world, the treasure is on the rightmost location.
Run this program and to see how the agent will improve its strategy of finding the treasure.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/
"""

import numpy as np
import pandas as pd
import time

np.random.seed(2)  # reproducible


N_STATES = 6   # the length of the 1 dimensional world
ACTIONS = ['left', 'right']     # available actions
EPSILON = 0.9   # greedy police
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 13   # maximum episodes
FRESH_TIME = 0.3    # fresh time for one move


def build_q_table(n_states, actions):
    q=np.zeros((n_states, len(actions)))
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table initial values
        columns=actions,    # actions's name
    )
    # print(table)    # show table
    return table


def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name


def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R


def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)


def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:

            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table


if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

----oT                          

In [1]:
"""
Reinforcement learning maze example.

Red rectangle:          explorer.
Black rectangles:       hells       [reward = -1].
Yellow bin circle:      paradise    [reward = +1].
All other states:       ground      [reward = 0].

This script is the main part which controls the update method of this example.
The RL is in RL_brain.py.

View more on my tutorial page: https://morvanzhou.github.io/tutorials/
"""

from maze_env import Maze
from RL_brain import QLearningTable


def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()

        while True:
            # fresh env
            env.render()

            # RL choose action based on observation
            action = RL.choose_action(str(observation))

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL learn from this transition
            RL.learn(str(observation), action, reward, str(observation_))

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break

    # end of game
    print('game over')
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    RL = QLearningTable(actions=list(range(env.n_actions)))

    env.after(100, update)
    env.mainloop()

f:\learn_python\rl_study\RL_brain.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
f:\learn_python\rl_study\RL_brain.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
f:\learn_python\rl_study\RL_brain.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
f:\learn_python\rl_study\RL_brain.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
f:\learn_python\rl_study\RL_brain.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use p